In [1]:
import sys
import torch
from torchvision import transforms

print('Python version:', sys.version)
sys.path.append('..')

from src.models.models import *
from src.models.aligner import *

Python version: 3.10.5 (main, Dec 17 2022, 19:41:24) [Clang 14.0.0 (clang-1400.0.29.202)]


In [2]:
sample_image = torch.rand(1, 3, 224, 224)
sample_test = ["Wlazl kotek na plotek i mruga"]
batch_sample_image = torch.rand(2, 3, 224, 224)
batch_sample_test = ["Wlazl kotek na plotek i mruga", "Wlazl kotek na plotek i mruga na ciebie"]

In [3]:
dino_family = DinoVisionModel
dino_family.SHAPES

{'dinov2_vits14': (384, 256),
 'dinov2_vitb14': (768, 256),
 'dinov2_vitl14': (1024, 256),
 'dinov2_vitg14': (1536, 256)}

In [4]:
base_dino = dino_family(checkpoint="dinov2_vitb14", pooling="cls")
base_dino

2023-12-05 12:56:53.016 | DEBUG    | src.models.base:__init__:16 - Model image DinoVisionModel initialized.
Using cache found in /Users/vladimirzaigrajew/.cache/torch/hub/facebookresearch_dinov2_main
xFormers not available
xFormers not available


DinoVisionModel(
  (_model): DinoVisionTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 768, kernel_size=(14, 14), stride=(14, 14))
      (norm): Identity()
    )
    (blocks): ModuleList(
      (0-11): 12 x NestedTensorBlock(
        (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (attn): MemEffAttention(
          (qkv): Linear(in_features=768, out_features=2304, bias=True)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=768, out_features=768, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (ls1): LayerScale()
        (drop_path1): Identity()
        (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (act): GELU(approximate='none')
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (drop): Dropout(p=0.0, inplace=False)
        

In [9]:
trans = transforms.Compose(base_dino.get_transformations()['image'])
trans(sample_image).shape, base_dino(sample_image).shape

/Users/vladimirzaigrajew/Documents/projects/ClipAligner/.venv/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


(torch.Size([1, 3, 224, 224]), torch.Size([1, 768]))

In [10]:
with torch.inference_mode():
    output_image = base_dino(trans(sample_image))
    output_batch_image = base_dino(trans(batch_sample_image))

In [11]:
output_image.shape, output_batch_image.shape, base_dino.output_dim

(torch.Size([1, 768]), torch.Size([2, 768]), 768)

In [12]:
with torch.inference_mode():
    for pooler in ["cls", "mean", "max", "mean_max", "cls_mean", "cls_max", "cls_mean_max"]:
        for checkpoint in ["dinov2_vits14", "dinov2_vitb14"]:#, "dinov2_vitl14", "dinov2_vitg14"]:
            dino = dino_family(checkpoint=checkpoint, pooling=pooler)
            print(f"pooler: {pooler}, checkpoint: {checkpoint}, output_dim: {dino.output_dim}, output_shape: {dino(sample_image).shape}, batch_output_shape: {dino(batch_sample_image).shape}")

2023-12-05 13:00:34.030 | DEBUG    | src.models.base:__init__:16 - Model image DinoVisionModel initialized.
Using cache found in /Users/vladimirzaigrajew/.cache/torch/hub/facebookresearch_dinov2_main
2023-12-05 13:00:35.766 | DEBUG    | src.models.base:__init__:16 - Model image DinoVisionModel initialized.


pooler: cls, checkpoint: dinov2_vits14, output_dim: 384, output_shape: torch.Size([1, 384]), batch_output_shape: torch.Size([2, 384])


Using cache found in /Users/vladimirzaigrajew/.cache/torch/hub/facebookresearch_dinov2_main
2023-12-05 13:00:40.476 | DEBUG    | src.models.base:__init__:16 - Model image DinoVisionModel initialized.


pooler: cls, checkpoint: dinov2_vitb14, output_dim: 768, output_shape: torch.Size([1, 768]), batch_output_shape: torch.Size([2, 768])


Using cache found in /Users/vladimirzaigrajew/.cache/torch/hub/facebookresearch_dinov2_main
2023-12-05 13:00:41.770 | DEBUG    | src.models.base:__init__:16 - Model image DinoVisionModel initialized.


...

pooler: cls_max, checkpoint: dinov2_vitb14, output_dim: 1024, output_shape: torch.Size([1, 1024]), batch_output_shape: torch.Size([2, 1024])


Using cache found in /Users/vladimirzaigrajew/.cache/torch/hub/facebookresearch_dinov2_main
2023-12-05 13:01:07.899 | DEBUG    | src.models.base:__init__:16 - Model image DinoVisionModel initialized.


pooler: cls_mean_max, checkpoint: dinov2_vits14, output_dim: 896, output_shape: torch.Size([1, 896]), batch_output_shape: torch.Size([2, 896])


Using cache found in /Users/vladimirzaigrajew/.cache/torch/hub/facebookresearch_dinov2_main


pooler: cls_mean_max, checkpoint: dinov2_vitb14, output_dim: 1280, output_shape: torch.Size([1, 1280]), batch_output_shape: torch.Size([2, 1280])


In [13]:
try:
    dino_family(checkpoint="dinov2_vits14", pooling="Wrong")
    assert False
except ValueError:
    assert True

2023-12-05 13:01:13.137 | DEBUG    | src.models.base:__init__:16 - Model image DinoVisionModel initialized.
Using cache found in /Users/vladimirzaigrajew/.cache/torch/hub/facebookresearch_dinov2_main


In [14]:
robert_family = RobertaCaptionModel
robert_family.SHAPES

{'roberta-base': 768, 'roberta-large': 1024, 'xlm-roberta-base': 250002}

In [15]:
base_robert = robert_family(checkpoint="roberta-base", pooling="cls")
base_robert

2023-12-05 13:01:14.299 | DEBUG    | src.models.base:__init__:32 - Model caption RobertaCaptionModel initialized.


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2023-12-05 13:01:17.692 | INFO     | src.models.models:__init__:84 - Sssssh don't listen to the warning above, Robert can be grumpy.


RobertaCaptionModel(
  (_model): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): 

In [17]:
tokenizer = base_robert.get_transformations()['caption']
tokenizer(sample_test, return_tensors="pt", padding=True).keys(), tokenizer(batch_sample_test, return_tensors="pt", padding=True).keys()

(dict_keys(['input_ids', 'attention_mask']),
 dict_keys(['input_ids', 'attention_mask']))

In [18]:
with torch.inference_mode():
    output_test = base_robert(tokenizer(sample_test, return_tensors="pt", padding=True))
    output_batch_test = base_robert(tokenizer(batch_sample_test, return_tensors="pt", padding=True))

In [19]:
output_test.shape, output_batch_test.shape, base_robert.output_dim

(torch.Size([1, 768]), torch.Size([2, 768]), 768)

In [20]:
for pooler in ["cls", "mean", "max", "mean_max", "cls_mean", "cls_max", "cls_mean_max"]:
    for checkpoint in ["roberta-base", "xlm-roberta-base"]:
        robert = robert_family(checkpoint=checkpoint, pooling=pooler)
        print(f"pooler: {pooler}, checkpoint: {checkpoint}, output_dim: {robert.output_dim}, output_shape: {robert(tokenizer(sample_test, return_tensors='pt', padding=True)).shape}, batch_output_shape: {robert(tokenizer(batch_sample_test, return_tensors='pt', padding=True)).shape}")

2023-12-05 13:03:01.020 | DEBUG    | src.models.base:__init__:32 - Model caption RobertaCaptionModel initialized.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2023-12-05 13:03:05.717 | INFO     | src.models.models:__init__:84 - Sssssh don't listen to the warning above, Robert can be grumpy.
2023-12-05 13:03:05.968 | DEBUG    | src.models.base:__init__:32 - Model caption RobertaCaptionModel initialized.


pooler: cls, checkpoint: roberta-base, output_dim: 768, output_shape: torch.Size([1, 768]), batch_output_shape: torch.Size([2, 768])


Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
2023-12-05 13:03:14.838 | INFO     | src.models.models:__init__:84 - Sssssh don't listen to the warning above, Robert can be grumpy.
2023-12-05 13:03:15.196 | DEBUG    | src.models.base:__init__:32 - Model caption RobertaCaptionModel initialized.


pooler: cls, checkpoint: xlm-roberta-base, output_dim: 250002, output_shape: torch.Size([1, 250002]), batch_output_shape: torch.Size([2, 250002])


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2023-12-05 13:03:17.688 | INFO     | src.models.models:__init__:84 - Sssssh don't listen to the warning above, Robert can be grumpy.
2023-12-05 13:03:18.600 | DEBUG    | src.models.base:__init__:32 - Model caption RobertaCaptionModel initialized.


...

pooler: cls_max, checkpoint: xlm-roberta-base, output_dim: 500004, output_shape: torch.Size([1, 500004]), batch_output_shape: torch.Size([2, 500004])


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2023-12-05 13:04:22.737 | INFO     | src.models.models:__init__:84 - Sssssh don't listen to the warning above, Robert can be grumpy.
2023-12-05 13:04:23.903 | DEBUG    | src.models.base:__init__:32 - Model caption RobertaCaptionModel initialized.


pooler: cls_mean_max, checkpoint: roberta-base, output_dim: 2304, output_shape: torch.Size([1, 2304]), batch_output_shape: torch.Size([2, 2304])


Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
2023-12-05 13:04:33.425 | INFO     | src.models.models:__init__:84 - Sssssh don't listen to the warning above, Robert can be grumpy.


pooler: cls_mean_max, checkpoint: xlm-roberta-base, output_dim: 750006, output_shape: torch.Size([1, 750006]), batch_output_shape: torch.Size([2, 750006])


In [21]:
try:
    robert_family(checkpoint="roberta-base", pooling="Wrong")
    assert False
except ValueError:
    assert True

2023-12-05 13:04:34.010 | DEBUG    | src.models.base:__init__:32 - Model caption RobertaCaptionModel initialized.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2023-12-05 13:04:37.535 | INFO     | src.models.models:__init__:84 - Sssssh don't listen to the warning above, Robert can be grumpy.


In [40]:
model = ClipAligner(
    DinoVisionModel(checkpoint="dinov2_vitb14", pooling="cls"),
    RobertaCaptionModel(checkpoint="roberta-base", pooling="cls"),
    vision_layer=[(768, "mlp"), (512, "mlp")],
    caption_layer=[(768, "mlp"), (512, "mlp")],
)

2023-12-05 13:12:05.764 | DEBUG    | src.models.base:__init__:16 - Model image DinoVisionModel initialized.
Using cache found in /Users/vladimirzaigrajew/.cache/torch/hub/facebookresearch_dinov2_main
2023-12-05 13:12:09.467 | DEBUG    | src.models.base:__init__:32 - Model caption RobertaCaptionModel initialized.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2023-12-05 13:12:11.966 | INFO     | src.models.models:__init__:84 - Sssssh don't listen to the warning above, Robert can be grumpy.
2023-12-05 13:12:11.968 | DEBUG    | src.models.base:__init__:50 - Model image-caption ClipAligner initialized.


In [41]:
out = model.get_transformations()
trans, tokenizer = out['image'], out['caption']
trans = transforms.Compose(trans)

In [42]:
with torch.inference_mode():
    output_test = model(trans(sample_image), tokenizer(sample_test, return_tensors="pt", padding=True))
    output_batch_test = model(trans(batch_sample_image), tokenizer(batch_sample_test, return_tensors="pt", padding=True))

/Users/vladimirzaigrajew/Documents/projects/ClipAligner/.venv/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


In [43]:
output_test[0].shape, output_batch_test[0].shape, output_test[1].shape, output_batch_test[1].shape, model.output_dim

(torch.Size([1, 512]),
 torch.Size([2, 512]),
 torch.Size([1, 512]),
 torch.Size([2, 512]),
 512)

In [44]:
with torch.inference_mode():
    for vision_layer, caption_layer in [
        ([(768, "mlp"), (512, "mlp")], [(768, "mlp"), (512, "mlp")]),
        ([(768, "conv"), (512, "conv")], [(768, "conv"), (512, "conv")]),
        ([(768, "res"), (512, "res")], [(768, "res"), (512, "res")]),
        ([(768, "res"), (512, "conv"), (256, "mlp")], [(768, "res"), (512, "conv"), (256, "mlp")]),
        ([(768, "conv"), (512, "mlp")], [(768, "res"), (512, "mlp")]),
    ]:
        for pooler in ["cls", "mean", "mean_max", "cls_mean_max"]:
            model = ClipAligner(
                DinoVisionModel(checkpoint="dinov2_vitb14", pooling=pooler),
                RobertaCaptionModel(checkpoint="roberta-base", pooling="cls"),
                vision_layer=vision_layer,
                caption_layer=caption_layer,
            )
            output = model(trans(sample_image), tokenizer(sample_test, return_tensors="pt", padding=True))
            print(f"pooler: {pooler}, vision_layer: {vision_layer}, caption_layer: {caption_layer}, output_dim: {model.output_dim}, output_shape_image: {output[0].shape}, output_shape_caption: {output[1].shape}")

2023-12-05 13:12:15.791 | DEBUG    | src.models.base:__init__:16 - Model image DinoVisionModel initialized.


Using cache found in /Users/vladimirzaigrajew/.cache/torch/hub/facebookresearch_dinov2_main
2023-12-05 13:12:19.865 | DEBUG    | src.models.base:__init__:32 - Model caption RobertaCaptionModel initialized.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2023-12-05 13:12:21.654 | INFO     | src.models.models:__init__:84 - Sssssh don't listen to the warning above, Robert can be grumpy.
2023-12-05 13:12:21.656 | DEBUG    | src.models.base:__init__:50 - Model image-caption ClipAligner initialized.
2023-12-05 13:12:22.135 | DEBUG    | src.models.base:__init__:16 - Model image DinoVisionModel initialized.


pooler: cls, vision_layer: [(768, 'mlp'), (512, 'mlp')], caption_layer: [(768, 'mlp'), (512, 'mlp')], output_dim: 512, output_shape_image: torch.Size([1, 512]), output_shape_caption: torch.Size([1, 512])


Using cache found in /Users/vladimirzaigrajew/.cache/torch/hub/facebookresearch_dinov2_main
2023-12-05 13:12:25.466 | DEBUG    | src.models.base:__init__:32 - Model caption RobertaCaptionModel initialized.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2023-12-05 13:12:27.327 | INFO     | src.models.models:__init__:84 - Sssssh don't listen to the warning above, Robert can be grumpy.
2023-12-05 13:12:27.328 | DEBUG    | src.models.base:__init__:50 - Model image-caption ClipAligner initialized.
2023-12-05 13:12:27.685 | DEBUG    | src.models.base:__init__:16 - Model image DinoVisionModel initialized.


pooler: mean, vision_layer: [(768, 'mlp'), (512, 'mlp')], caption_layer: [(768, 'mlp'), (512, 'mlp')], output_dim: 512, output_shape_image: torch.Size([1, 512]), output_shape_caption: torch.Size([1, 512])


...

pooler: mean, vision_layer: [(768, 'conv'), (512, 'mlp')], caption_layer: [(768, 'res'), (512, 'mlp')], output_dim: 512, output_shape_image: torch.Size([1, 512]), output_shape_caption: torch.Size([1, 512])


Using cache found in /Users/vladimirzaigrajew/.cache/torch/hub/facebookresearch_dinov2_main
2023-12-05 13:13:49.026 | DEBUG    | src.models.base:__init__:32 - Model caption RobertaCaptionModel initialized.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2023-12-05 13:13:50.954 | INFO     | src.models.models:__init__:84 - Sssssh don't listen to the warning above, Robert can be grumpy.
2023-12-05 13:13:50.968 | DEBUG    | src.models.base:__init__:50 - Model image-caption ClipAligner initialized.
2023-12-05 13:13:51.357 | DEBUG    | src.models.base:__init__:16 - Model image DinoVisionModel initialized.


pooler: mean_max, vision_layer: [(768, 'conv'), (512, 'mlp')], caption_layer: [(768, 'res'), (512, 'mlp')], output_dim: 512, output_shape_image: torch.Size([1, 512]), output_shape_caption: torch.Size([1, 512])


Using cache found in /Users/vladimirzaigrajew/.cache/torch/hub/facebookresearch_dinov2_main
2023-12-05 13:13:53.710 | DEBUG    | src.models.base:__init__:32 - Model caption RobertaCaptionModel initialized.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2023-12-05 13:13:55.479 | INFO     | src.models.models:__init__:84 - Sssssh don't listen to the warning above, Robert can be grumpy.
2023-12-05 13:13:55.480 | DEBUG    | src.models.base:__init__:50 - Model image-caption ClipAligner initialized.


pooler: cls_mean_max, vision_layer: [(768, 'conv'), (512, 'mlp')], caption_layer: [(768, 'res'), (512, 'mlp')], output_dim: 512, output_shape_image: torch.Size([1, 512]), output_shape_caption: torch.Size([1, 512])


In [45]:

for layer_vision, layer_caption in [["mlp", "Wrong"], ["Wrong", "mlp"]]:
    try:
        model = ClipAligner(
            DinoVisionModel(checkpoint="dinov2_vitb14", pooling="cls"),
            RobertaCaptionModel(checkpoint="roberta-base", pooling="cls"),
            vision_layer=[(768, layer_vision), (512, layer_vision)],
            caption_layer=[(768, layer_caption), (512, layer_caption)],
        )
        assert False
    except ValueError:
        assert True

2023-12-05 13:13:55.861 | DEBUG    | src.models.base:__init__:16 - Model image DinoVisionModel initialized.
Using cache found in /Users/vladimirzaigrajew/.cache/torch/hub/facebookresearch_dinov2_main
2023-12-05 13:13:58.311 | DEBUG    | src.models.base:__init__:32 - Model caption RobertaCaptionModel initialized.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2023-12-05 13:14:00.071 | INFO     | src.models.models:__init__:84 - Sssssh don't listen to the warning above, Robert can be grumpy.
2023-12-05 13:14:00.072 | DEBUG    | src.models.base:__init__:50 - Model image-caption ClipAligner initialized.
2023-12-05 13:14:00.215 | DEBUG    | src.models.base:__init__:16 - Model image DinoVisionModel initialized.
Using cache found in /Users/vladimirzaigra

In [46]:
from src.models.factory import create_model

In [1]:
model = create_model(
    {
        "image": {"name": "dinov2", "checkpoint": "dinov2_vitb14", "pooling": "cls"},
        "caption": {"name": "roberta", "checkpoint": "roberta-base", "pooling": "cls"},
        "alignment": {"name": "clip_aligner",
                      "vision_layer": [{"name": "mlp", "output": 768, "activation": "tanh"}, {"name": "mlp", "output": 512, "activation": ""}],
                      "caption_layer": [{"name": "mlp", "output": 768, "activation": "tanh"}, {"name": "mlp", "output": 512, "activation": ""}], 
        },
    }
)

NameError: name 'create_model' is not defined

In [50]:
out = model.get_transformations()
trans, tokenizer = out['image'], out['caption']
trans = transforms.Compose(trans)

In [51]:
with torch.inference_mode():
    output_test = model(trans(sample_image), tokenizer(sample_test, return_tensors="pt", padding=True))
    output_batch_test = model(trans(batch_sample_image), tokenizer(batch_sample_test, return_tensors="pt", padding=True))

/Users/vladimirzaigrajew/Documents/projects/ClipAligner/.venv/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


In [52]:
output_test[0].shape, output_batch_test[0].shape, output_test[1].shape, output_batch_test[1].shape, model.output_dim

(torch.Size([1, 512]),
 torch.Size([2, 512]),
 torch.Size([1, 512]),
 torch.Size([2, 512]),
 512)